In [2]:
from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import json
import networkx
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous, no_more_discontiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
import pandas
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from functools import partial
import gerrychain.tree_proposals as tp
from gerrychain.constraints.validity import within_percent_of_ideal_population
import matplotlib.pyplot as plt
import gerrychain

#NOTE: must have tqdm installed (pip install tqdm in terminal)

In [3]:
#NOTE: An island warning will pop up.
graph = Graph.from_file("/Users/caranix/Documents/MGGG/AK_precincts/alaska_precincts.shp")
print("graph done")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/graph/graph.py:196: UserWarning: Found islands (degree-0 nodes). Indices of islands: {352, 394, 372, 351}
  "Found islands (degree-0 nodes). Indices of islands: {}".format(islands)


graph done


In [22]:
election = gerrychain.Election("AK", ["PRES16", "USH14", "SEN16","GOV18","USH16","USH18"])

In [15]:
updaters = {"population": gerrychain.updaters.Tally("POPULATION", alias="population"), "AK": election, "cut_edges": gerrychain.updaters.cut_edges}

# Create an initial partition
initial_partition  = gerrychain.GeographicPartition(graph, assignment="HDIST",updaters= updaters)
#NOTE: assignment based on House Districts


In [16]:
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(tp.recom,
                   pop_col="POPULATION",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )

In [17]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

In [18]:
chain = gerrychain.MarkovChain(
    proposal=propose_random_flip,
    constraints=[constraints.within_percent_of_ideal_population(initial_partition, .03), no_more_discontiguous, compactness_bound],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=1000
)
#THIS CHAIN IS NOW WORKING!! (but it isn't based on proposal, but propose_random_flip)


In [23]:
#getting a sketchy key error here...
chain_results = {"PRES16": [],
                 "USH14": [],
                 "SEN16": [],
                 "SEN18": [],
                 "USH16": [], 
                 "USH18": []}

for partition in chain:
    for key, _ in chain_results.items():
        chain_results[key].append(sorted(partition["AK"].percents(key))) 


KeyError: 'PRES16'

In [ ]:
chain_dfs = {"PRES16": [],
             "USH14": [],
             "SEN16": [],
             "SEN18": [],
             "USH16": [], 
             "USH18": []}

for key, val in chain_results.items():
    chain_dfs[key] = pandas.DataFrame(val)

In [ ]:
#data = json.dumps(networkx.readwrite.json_graph.adjacency_data(graph))
#It won't let me save graph due to a Polygon error- not a major issue for me atm 
#with open('/Users/caranix/Documents/alaska_graphAK_DATA.json') as f:
#        data = json.load(f)
#graph = networkx.readwrite.json_graph.adjacency_graph(data)
#this is for when I can fix saving the grpah as a .json